# Pruebas gemma

In [49]:
import pandas as pd

## Calidad de generación del texto: BLEU

In [2]:
import evaluate

/home/raulmartinez/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def calcular_bleu(df):
    predictions = df["output"]
    references = df["input"]
    bleu = evaluate.load ("bleu")
    results = bleu.compute (predictions = predictions, references = references)
    print (results)

In [5]:
df_gemma_sin_rol_1 = pd.read_csv("./analysis/gemma_sin_rol_1.csv")
df_gemma_con_rol_1 = pd.read_csv("./analysis/gemma_con_rol_1.csv")
df_gemma_sin_rol_0_5 = pd.read_csv("./analysis/gemma_sin_rol_0_5.csv")
df_gemma_con_rol_0_5 = pd.read_csv("./analysis/gemma_con_rol_0_5.csv")
df_gemma_sin_rol_0_1 = pd.read_csv("./analysis/gemma_sin_rol_0_1.csv")
df_gemma_con_rol_0_1 = pd.read_csv("./analysis/gemma_con_rol_0_1.csv")

print("Gemma sin rol 1")
calcular_bleu(df_gemma_sin_rol_1)
print("Gemma con rol 1")
calcular_bleu(df_gemma_con_rol_1)
print("Gemma sin rol 0.5")
calcular_bleu(df_gemma_sin_rol_0_5)
print("Gemma con rol 0.5")
calcular_bleu(df_gemma_con_rol_0_5)
print("Gemma sin rol 0.1")
calcular_bleu(df_gemma_sin_rol_0_1)
print("Gemma con rol 0.1")
calcular_bleu(df_gemma_con_rol_0_1)

Gemma sin rol 1
{'bleu': 2.5562811827773874e-07, 'precisions': [0.7607627572116554, 0.26030283726172393, 0.10263118921854174, 0.05803948873526633], 'brevity_penalty': 1.3793543596593262e-06, 'length_ratio': 0.06899456622062017, 'translation_length': 20557, 'reference_length': 297951}
Gemma con rol 1
{'bleu': 1.567980944699892e-07, 'precisions': [0.7672171555336456, 0.2557238639185498, 0.0955716787590693, 0.05142425006301991], 'brevity_penalty': 8.898034684187848e-07, 'length_ratio': 0.06696907570460316, 'translation_length': 20285, 'reference_length': 302901}
Gemma sin rol 0.5
{'bleu': 2.631053367949994e-07, 'precisions': [0.7497828814049986, 0.24305015552099535, 0.09125624204445315, 0.04961530873939633], 'brevity_penalty': 1.5523949872243844e-06, 'length_ratio': 0.06956177358021956, 'translation_length': 20726, 'reference_length': 297951}
Gemma con rol 0.5
{'bleu': 1.3917434414003416e-07, 'precisions': [0.7550899093475999, 0.2517342915606129, 0.09076280987579827, 0.04635962912296702],

## Comparación entre pruebas con rol y sin rol a diferentes temperaturas

In [6]:
from evaluate import load

### Bert por defecto

In [28]:
# Modelo Bert por defecto
bertscore = load("bertscore")

import numpy as np

In [29]:
def calcular_bertscore_defecto(df_sin_rol, df_con_rol):
    predictions = df_sin_rol["output"]
    references = df_con_rol["output"]
    results = bertscore.compute(predictions=predictions, references=references, lang="en", model_type="distilbert-base-uncased")
    print("Precision:", sum(results["precision"]) / len(results["precision"]))
    print("Recall:", sum(results["recall"]) / len(results["recall"]))
    print("F1:", sum(results["f1"]) / len(results["f1"]))
    df = pd.DataFrame({
    "precision": [np.mean(results["precision"])],
    "recall": [np.mean(results["recall"])],
    "f1": [np.mean(results["f1"])]
})
    return df

In [30]:
print ("BertScore por defecto a temperatura 1")
calcular_bertscore_defecto(df_gemma_sin_rol_1, df_gemma_con_rol_1)
print ("BertScore por defecto a temperatura 0.5")
calcular_bertscore_defecto(df_gemma_sin_rol_0_5, df_gemma_con_rol_0_5)
print ("BertScore por defecto a temperatura 0.1")
calcular_bertscore_defecto(df_gemma_sin_rol_0_1, df_gemma_con_rol_0_1)

BertScore por defecto a temperatura 1
Precision: 0.9005568917592367
Recall: 0.9007785137494405
F1: 0.9005949890613556
BertScore por defecto a temperatura 0.5
Precision: 0.8772079618771871
Recall: 0.8757964762051901
F1: 0.876428058942159
BertScore por defecto a temperatura 0.1
Precision: 0.8983016264438629
Recall: 0.8984694230556488
F1: 0.8983149083455404


,precision,recall,f1
0,0.898302,0.898469,0.898315


### Bert utilizando modelo Bio_ClinicalBERT

In [11]:
from bert_score import score

In [37]:
def calcular_bertscore_bioclinical(df_sin_rol, df_con_rol):
    predictions = df_sin_rol["output"].astype(str).tolist()
    references = df_con_rol["output"].astype(str).tolist()

    P, R, F1 = score(predictions, references, lang="en", num_layers=12, idf=False, model_type="emilyalsentzer/Bio_ClinicalBERT")
    print(f"Precission: {sum(P) / len(P)}")
    print(f"Recall: {sum(R) / len(R)}")
    print(f"F1: {sum(F1) / len(F1)}")

    df = pd.DataFrame({
        "precision": [sum(P) / len(P)],
        "recall": [sum(R) / len(R)],
        "f1": [sum(F1) / len(F1)]
    })
    return df

    

In [38]:
print ("BertScore con modelo Bio_ClinicalBERT a temperatura 1")
calcular_bertscore_bioclinical(df_gemma_sin_rol_1, df_gemma_con_rol_1)
print ("BertScore con modelo Bio_ClinicalBERT a temperatura 0.5")
calcular_bertscore_bioclinical(df_gemma_sin_rol_0_5, df_gemma_con_rol_0_5)
print ("BertScore con modelo Bio_ClinicalBERT a temperatura 0.1")
calcular_bertscore_bioclinical(df_gemma_sin_rol_0_1, df_gemma_con_rol_0_1)

BertScore con modelo Bio_ClinicalBERT a temperatura 1
Precission: 0.872083306312561
Recall: 0.8703686594963074
F1: 0.8710890412330627
BertScore con modelo Bio_ClinicalBERT a temperatura 0.5
Precission: 0.838509738445282
Recall: 0.837006688117981
F1: 0.8376386165618896
BertScore con modelo Bio_ClinicalBERT a temperatura 0.1
Precission: 0.8701820969581604
Recall: 0.8672230839729309
F1: 0.8686009645462036


,precision,recall,f1
0,tensor(0.8702),tensor(0.8672),tensor(0.8686)


## Medidas de readabilidad y comprensión del texto

In [ ]:
# pip install https://github.com/andreasvc/readability/tarball/master

In [16]:
import re
import nltk
from nltk.probability import FreqDist
from textblob import TextBlob
import readability
import spacy

In [39]:
def evaluate_prompt(prompt_list,max_rows=5):
    # Correctness (basic grammar check with spaCy)

    flesh_scores = []
    smog_scores = []
    dale_scores = []
    sentiment_polarity_scores = []
    sentiment_subjectivity_scores = []
    lexical_diversity_scores = []
    lexical_richness_scores = []

    for prompt_element in prompt_list[:max_rows]:
        prompt = prompt_element['text']

        # Clarity (readability score using Readability)
        results_r = readability.getmeasures(prompt, lang='en')
        flesch_reading_ease = results_r['readability grades']['FleschReadingEase']
        flesh_scores.append(flesch_reading_ease)

        flesch_reading_ease = results_r['readability grades']['FleschReadingEase']
        smog_index = results_r['readability grades']['SMOGIndex']
        smog_scores.append(smog_index)


        dale_chall_score = results_r['readability grades']['DaleChallIndex']
        dale_scores.append(dale_chall_score)

        # Engagement (sentiment analysis with TextBlob)
        blob = TextBlob(prompt)
        sentiment_polarity_scores.append(blob.sentiment.polarity)
        sentiment_subjectivity_scores.append(blob.sentiment.subjectivity)

        words = prompt.split()
        unique_words = set(words)
        lexical_diversity = len(unique_words) / len(words) if words else 0
        lexical_diversity_scores.append(lexical_diversity)

        # Lexical richness (frequency distribution of words)
        fdist = FreqDist(words)
        lexical_richness = sum(fdist.freq(word) for word in unique_words) / len(unique_words) if unique_words else 0
        lexical_richness_scores.append(lexical_richness)

    return flesh_scores, smog_scores, dale_scores, sentiment_polarity_scores, sentiment_subjectivity_scores,lexical_diversity_scores,lexical_richness_scores

  #'], engagement_scores["sentiment_polarity"], engagement_scores["sentiment_subjectivity"], engagement_scores["lexical_diversity"], engagement_scores["lexical_richness"]

In [40]:
def calcular_metricas_comprension(df):
    prompt_list = [{'text': output} for output in df['output']]
    flesh_scores, smog_scores, dale_scores, sentiment_polarity_scores, sentiment_subjectivity_scores, lexical_diversity_scores, lexical_richness_scores = evaluate_prompt(prompt_list)

    print("Flesch Reading Ease Scores:", sum(flesh_scores) / len(flesh_scores))
    print("SMOG Index Scores:", sum(smog_scores) / len(smog_scores))
    print("Dale-Chall Index Scores:", sum(dale_scores) / len(dale_scores))
    print("Sentiment Polarity Scores:", sum(sentiment_polarity_scores) / len(sentiment_polarity_scores))
    print("Sentiment Subjectivity Scores:", sum(sentiment_subjectivity_scores) / len(sentiment_subjectivity_scores))
    print("Lexical Diversity Scores:", sum(lexical_diversity_scores) / len(lexical_diversity_scores))
    print("Lexical Richness Scores:", sum(lexical_richness_scores) / len(lexical_richness_scores))

    df = pd.DataFrame({
        "flesch_reading_ease": [np.mean(flesh_scores)],
        "smog_index": [np.mean(smog_scores)],
        "dale_chall_index": [np.mean(dale_scores)],
        "sentiment_polarity": [np.mean(sentiment_polarity_scores)],
        "sentiment_subjectivity": [np.mean(sentiment_subjectivity_scores)],
        "lexical_diversity": [np.mean(lexical_diversity_scores)],
        "lexical_richness": [np.mean(lexical_richness_scores)]
    })
    return df

In [41]:
print("Metricas de readabilty y comprensión para sin rol a 1")
calcular_metricas_comprension(df_gemma_sin_rol_1)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para con rol a 1")
calcular_metricas_comprension(df_gemma_con_rol_1)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para sin rol a 0.5")
calcular_metricas_comprension(df_gemma_sin_rol_0_5)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para con rol a 0.5")
calcular_metricas_comprension(df_gemma_con_rol_0_5)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para sin rol a 0.1")
calcular_metricas_comprension(df_gemma_sin_rol_0_1)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para con rol a 0.1")
calcular_metricas_comprension(df_gemma_con_rol_0_1)

Metricas de readabilty y comprensión para sin rol a 1
Flesch Reading Ease Scores: 34.22234708982448
SMOG Index Scores: 13.058184468534472
Dale-Chall Index Scores: 7.898114446416173
Sentiment Polarity Scores: 0.022286375661375663
Sentiment Subjectivity Scores: 0.3416752645502646
Lexical Diversity Scores: 0.7564128854920904
Lexical Richness Scores: 0.014400490012946335
----------------------------------------------------
Metricas de readabilty y comprensión para con rol a 1
Flesch Reading Ease Scores: 31.6450007348268
SMOG Index Scores: 15.185644137160375
Dale-Chall Index Scores: 8.598488592455155
Sentiment Polarity Scores: 0.0637936507936508
Sentiment Subjectivity Scores: 0.5060899470899471
Lexical Diversity Scores: 0.7376177487147453
Lexical Richness Scores: 0.014315781683464559
----------------------------------------------------
Metricas de readabilty y comprensión para sin rol a 0.5
Flesch Reading Ease Scores: 35.348670973085504
SMOG Index Scores: 13.076315167439983
Dale-Chall Index

,flesch_reading_ease,smog_index,dale_chall_index,sentiment_polarity,sentiment_subjectivity,lexical_diversity,lexical_richness
0,34.11296,13.563943,8.029515,0.056196,0.383519,0.730298,0.014879


## Calcular longitud media de los textos

In [23]:
# Calcular longitud media de los textos por palabras
def calculate_average_word_count(text_list):
    total_words = sum(len(text.split()) for text in text_list)
    average_word_count = total_words / len(text_list)
    return average_word_count

In [42]:
def calcular_longitud_texto_y_respuesta(df):
    longitud_textos = calculate_average_word_count(df['input'])
    longitud_respuestas = calculate_average_word_count(df['output'])

    print(f"Longitud de los textos: {longitud_textos}")
    print(f"Longitud de las respuestas: {longitud_respuestas}")

    df = pd.DataFrame({
        "longitud_textos": [longitud_textos],
        "longitud_respuestas": [longitud_respuestas]
    })
    return df

In [43]:
print("Longitud textos y respuestas sin rol a 1")
calcular_longitud_texto_y_respuesta(df_gemma_sin_rol_1)
print("----------------------------------------------------")
print("Longitud textos y respuestas con rol a 1")
calcular_longitud_texto_y_respuesta(df_gemma_con_rol_1)
print("----------------------------------------------------")
print("Longitud textos y respuestas sin rol a 0.5")
calcular_longitud_texto_y_respuesta(df_gemma_sin_rol_0_5)
print("----------------------------------------------------")
print("Longitud textos y respuestas con rol a 0.5")
calcular_longitud_texto_y_respuesta(df_gemma_con_rol_0_5)
print("----------------------------------------------------")
print("Longitud textos y respuestas sin rol a 0.1")
calcular_longitud_texto_y_respuesta(df_gemma_sin_rol_0_1)
print("----------------------------------------------------")
print("Longitud textos y respuestas con rol a 0.1")
calcular_longitud_texto_y_respuesta(df_gemma_con_rol_0_1)

Longitud textos y respuestas sin rol a 1
Longitud de los textos: 1440.46
Longitud de las respuestas: 95.02
----------------------------------------------------
Longitud textos y respuestas con rol a 1
Longitud de los textos: 1458.46
Longitud de las respuestas: 94.96666666666667
----------------------------------------------------
Longitud textos y respuestas sin rol a 0.5
Longitud de los textos: 1440.46
Longitud de las respuestas: 95.31333333333333
----------------------------------------------------
Longitud textos y respuestas con rol a 0.5
Longitud de los textos: 1458.46
Longitud de las respuestas: 94.72
----------------------------------------------------
Longitud textos y respuestas sin rol a 0.1
Longitud de los textos: 1440.46
Longitud de las respuestas: 95.18666666666667
----------------------------------------------------
Longitud textos y respuestas con rol a 0.1
Longitud de los textos: 1458.46
Longitud de las respuestas: 94.60666666666667


,longitud_textos,longitud_respuestas
0,1458.46,94.606667


# Pruebas llama

## Calidad de generación del texto: BLEU

In [44]:
df_llama_sin_rol_1 = pd.read_csv("./analysis/llama_sin_rol_1.csv")
df_llama_con_rol_1 = pd.read_csv("./analysis/llama_con_rol_1.csv")
df_llama_sin_rol_0_5 = pd.read_csv("./analysis/llama_sin_rol_0_5.csv")
df_llama_con_rol_0_5 = pd.read_csv("./analysis/llama_con_rol_0_5.csv")
df_llama_sin_rol_0_1 = pd.read_csv("./analysis/llama_sin_rol_0_1.csv")
df_llama_con_rol_0_1 = pd.read_csv("./analysis/llama_con_rol_0_1.csv")


print("Llama sin rol 1")
calcular_bleu(df_llama_sin_rol_1)
print("Llama con rol 1")
calcular_bleu(df_llama_con_rol_1)
print("Llama sin rol 0.5")
calcular_bleu(df_llama_sin_rol_0_5)
print("Llama con rol 0.5")
calcular_bleu(df_llama_con_rol_0_5)
print("Llama sin rol 0.1")
calcular_bleu(df_llama_sin_rol_0_1)
print("Llama con rol 0.1")
calcular_bleu(df_llama_con_rol_0_1)

Llama sin rol 1
{'bleu': 2.756384901940513e-07, 'precisions': [0.7622970680881996, 0.260678545277032, 0.10508974674207032, 0.061332672776814465], 'brevity_penalty': 1.4570331028759703e-06, 'length_ratio': 0.06925635423274297, 'translation_length': 20635, 'reference_length': 297951}
Llama con rol 1
{'bleu': 3.219687978006532e-07, 'precisions': [0.8143569393569393, 0.43202091664103354, 0.2692705104360405, 0.20207226908257836], 'brevity_penalty': 8.655855826818002e-07, 'length_ratio': 0.06684554720099574, 'translation_length': 19656, 'reference_length': 294051}
Llama sin rol 0.5
{'bleu': 5.7681761830072085e-08, 'precisions': [0.8277514691618646, 0.4618748920987512, 0.3053926975097231, 0.24231422380980266], 'brevity_penalty': 1.4064459583071183e-07, 'length_ratio': 0.059605306562467056, 'translation_length': 17527, 'reference_length': 294051}
Llama con rol 0.5
{'bleu': 3.645467783442035e-07, 'precisions': [0.83871459805766, 0.4749705385048932, 0.3132131977074405, 0.24384659416141957], 'bre

## Comparación entre pruebas con rol y sin rol a diferentes temperaturas

### Bert por defecto

In [45]:
print ("BertScore por defecto a temperatura 1")
calcular_bertscore_defecto(df_llama_sin_rol_1, df_llama_con_rol_1)
print ("BertScore por defecto a temperatura 0.5")
calcular_bertscore_defecto(df_llama_sin_rol_0_5, df_llama_con_rol_0_5)
print ("BertScore por defecto a temperatura 0.1")
calcular_bertscore_defecto(df_llama_sin_rol_0_1, df_llama_con_rol_0_1)

BertScore por defecto a temperatura 1
Precision: 0.8054464050134023
Recall: 0.8010353072484334
F1: 0.8030827442804972
BertScore por defecto a temperatura 0.5
Precision: 0.7990198000272115
Recall: 0.8010387722651163
F1: 0.7997263622283936
BertScore por defecto a temperatura 0.1
Precision: 0.7932461086908976
Recall: 0.7972064991792043
F1: 0.794953336318334


,precision,recall,f1
0,0.793246,0.797206,0.794953


### Bert utilizando modelo Bio_ClinicalBERT

In [46]:
print ("BertScore con modelo Bio_ClinicalBERT a temperatura 1")
calcular_bertscore_bioclinical(df_llama_sin_rol_1, df_llama_con_rol_1)
print ("BertScore con modelo Bio_ClinicalBERT a temperatura 0.5")
calcular_bertscore_bioclinical(df_llama_sin_rol_0_5, df_llama_con_rol_0_5)
print ("BertScore con modelo Bio_ClinicalBERT a temperatura 0.1")
calcular_bertscore_bioclinical(df_llama_sin_rol_0_1, df_llama_con_rol_0_1)

BertScore con modelo Bio_ClinicalBERT a temperatura 1
Precission: 0.7426342368125916
Recall: 0.7288378477096558
F1: 0.7354907393455505
BertScore con modelo Bio_ClinicalBERT a temperatura 0.5
Precission: 0.7294520139694214
Recall: 0.7297423481941223
F1: 0.7293215394020081
BertScore con modelo Bio_ClinicalBERT a temperatura 0.1
Precission: 0.723676323890686
Recall: 0.7207764983177185
F1: 0.7219652533531189


,precision,recall,f1
0,tensor(0.7237),tensor(0.7208),tensor(0.7220)


## Medidas de readabilidad y comprensión del texto

In [47]:
print("Metricas de readabilty y comprensión para sin rol a 1")
calcular_metricas_comprension(df_llama_sin_rol_1)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para con rol a 1")
calcular_metricas_comprension(df_llama_con_rol_1)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para sin rol a 0.5")
calcular_metricas_comprension(df_llama_sin_rol_0_5)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para con rol a 0.5")
calcular_metricas_comprension(df_llama_con_rol_0_5)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para sin rol a 0.1")
calcular_metricas_comprension(df_llama_sin_rol_0_1)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para con rol a 0.1")
calcular_metricas_comprension(df_llama_con_rol_0_1)

Metricas de readabilty y comprensión para sin rol a 1
Flesch Reading Ease Scores: 35.73721710962494
SMOG Index Scores: 13.092083281269657
Dale-Chall Index Scores: 7.97569182076518
Sentiment Polarity Scores: 0.0727435064935065
Sentiment Subjectivity Scores: 0.37562229437229433
Lexical Diversity Scores: 0.7285361261743131
Lexical Richness Scores: 0.015097966041170913
----------------------------------------------------
Metricas de readabilty y comprensión para con rol a 1
Flesch Reading Ease Scores: 39.55900010851506
SMOG Index Scores: 10.179524572218234
Dale-Chall Index Scores: 6.11039722890901
Sentiment Polarity Scores: -0.027253787878787888
Sentiment Subjectivity Scores: 0.19066693722943723
Lexical Diversity Scores: 0.7522334319075863
Lexical Richness Scores: 0.015713019960498235
----------------------------------------------------
Metricas de readabilty y comprensión para sin rol a 0.5
Flesch Reading Ease Scores: 39.230504234887384
SMOG Index Scores: 13.87000152930687
Dale-Chall Inde

,flesch_reading_ease,smog_index,dale_chall_index,sentiment_polarity,sentiment_subjectivity,lexical_diversity,lexical_richness
0,56.390815,9.460258,6.876006,-0.020694,0.242896,0.821884,0.015099


## Calcular longitud media de los textos

In [48]:
print("Longitud textos y respuestas sin rol a 1")
calcular_longitud_texto_y_respuesta(df_llama_sin_rol_1)
print("----------------------------------------------------")
print("Longitud textos y respuestas con rol a 1")
calcular_longitud_texto_y_respuesta(df_llama_con_rol_1)
print("----------------------------------------------------")
print("Longitud textos y respuestas sin rol a 0.5")
calcular_longitud_texto_y_respuesta(df_llama_sin_rol_0_5)
print("----------------------------------------------------")
print("Longitud textos y respuestas con rol a 0.5")
calcular_longitud_texto_y_respuesta(df_llama_con_rol_0_5)
print("----------------------------------------------------")
print("Longitud textos y respuestas sin rol a 0.1")
calcular_longitud_texto_y_respuesta(df_llama_sin_rol_0_1)
print("----------------------------------------------------")
print("Longitud textos y respuestas con rol a 0.1")
calcular_longitud_texto_y_respuesta(df_llama_con_rol_0_1)

Longitud textos y respuestas sin rol a 1
Longitud de los textos: 1440.46
Longitud de las respuestas: 95.52
----------------------------------------------------
Longitud textos y respuestas con rol a 1
Longitud de los textos: 1438.46
Longitud de las respuestas: 80.43333333333334
----------------------------------------------------
Longitud textos y respuestas sin rol a 0.5
Longitud de los textos: 1438.46
Longitud de las respuestas: 85.68666666666667
----------------------------------------------------
Longitud textos y respuestas con rol a 0.5
Longitud de los textos: 1438.46
Longitud de las respuestas: 79.17333333333333
----------------------------------------------------
Longitud textos y respuestas sin rol a 0.1
Longitud de los textos: 1438.46
Longitud de las respuestas: 87.4
----------------------------------------------------
Longitud textos y respuestas con rol a 0.1
Longitud de los textos: 1438.46
Longitud de las respuestas: 81.4


,longitud_textos,longitud_respuestas
0,1438.46,81.4
